[코알라] 주식 종료 가격 예측 경진대회 Stage 6 : 모델 후처리 및 마무리

안녕하세요. 데이크루 4기 코알라🐨 팀입니다. 

저희는 📈주식 종료 가격 예측 경진대회를 주제로 PBL를 수행합니다.

이번 활동을 통해 논리적인 접근 방식으로 모든 문제를 풀어갈 수 있는 데이커가 되는 것을 최종 목표로 하고 있습니다. 

Stage 6은 PBL을 구성하는 6가지 단계 중 마지막 단계로서 모델 후처리 및 마무리 내용을 담고있습니다.

다음의 포스팅은 데이크루 4기 활동으로 인하여 작성되었음을 알려드립니다.





# Stage 6.
# 1. Review

Stage 6에 들어가려면 Stage 5까지의 모든 코드를 입력해야지만 가능합니다.

간단하게 코드를 복습한다는 마음으로 천천히 코드를 읽어보며, 아래 코드를 실행시켜 주세요!




In [ ]:
import micropip
await micropip.install('requests')

In [3]:
import pandas as pd
import numpy as np
import datetime
import requests as req
import lightgbm as lgb
from sklearn import linear_model
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.feature_selection import SelectKBest, f_regression, f_classif
from sklearn import metrics
import os
import sys
import random
import warnings

warnings.filterwarnings(action='ignore')
os.chdir('./practice_data')

## Public, Private 예측 기준 일자
pub_base_dt = '20211029'  
pri_base_dt = '20211126'  

## Public, Private 평가 기간
dl_predict_pub_day = ['2021-11-01', '2021-11-02', '2021-11-03', '2021-11-04', '2021-11-05']   
dl_predict_pri_day = ['2021-11-29', '2021-11-30', '2021-12-01', '2021-12-02', '2021-12-03']   


## 과거 상장폐지종목 추출 
df_out_tt1 = pd.read_csv('코스피상폐현황.csv', encoding='utf-8') 
df_out_tt2 = pd.read_csv('코스닥상폐현황.csv', encoding='utf-8') 

df_out_tt1['market'] = 'KOSPI'
df_out_tt2['market'] = 'KOSDAQ'
df_out_tt = pd.concat([df_out_tt1, df_out_tt2])
df_out_tt.rename(columns = {"종목코드":"code", "회사명":"code_nm", "폐지일자":"out_dt", "폐지사유":"out_desc"}, inplace = True)
df_out_tt['code'] = df_out_tt['code'].astype(str) 
df_out_tt['code'] = df_out_tt['code'].str.zfill(6) 
df_out_tt['out_dt'] = pd.to_datetime(df_out_tt['out_dt'])
df_out_tt = df_out_tt[["code", "code_nm", "out_dt", "out_desc", "market"]]

## 현재 상장종목 전체 추출 : FinanceDataReader
df_in_tt = pd.read_csv('종목리스트.csv', encoding='utf-8')
df_in_tt.columns = ['code', 'isu_cd', 'code_nm', 'market', 'dept', 'close', 'changecode', 'changes', 'chagesratio', 
                    'open', 'high', 'low', 'volume', 'amount', 'marcap', 'stocks', 'marketId']
df_in_tt = df_in_tt[(df_in_tt['market'] !='KONEX') &                
                    (df_in_tt.code.str.startswith('5') != True) &   
                    (df_in_tt.code.str.startswith('6') != True) &   
                    (df_in_tt.code.str.startswith('7') != True) &   
                    (df_in_tt.code.str.len() == 6)]
df_in_tt['in_yn'] = 1  
df_in_tt['code'] = df_in_tt['code'].astype(str) 
df_in_tt['code'] = df_in_tt['code'].str.zfill(6) 
df_in_tt['close'] = df_in_tt['close'].astype(str)
df_in_tt['changecode'] = df_in_tt['close'].astype(str)

## 과거 상장폐지종목 + 현재 상장종목 결합
end_dt = datetime.datetime.now().strftime('%Y%m%d')   
df_stock_code = pd.merge(df_out_tt, df_in_tt, how='outer', on='code', suffixes=('_old', '_new'))
df_stock_code['in_yn'] = np.where(df_stock_code['in_yn'].notnull(), df_stock_code['in_yn'], 0)
df_stock_code['market'] = np.where(df_stock_code['market_new'].notnull(), df_stock_code['market_new'], df_stock_code['market_old'])
df_stock_code['code_nm'] = np.where(df_stock_code['code_nm_new'].notnull(), df_stock_code['code_nm_new'], df_stock_code['code_nm_old'])
df_stock_code['base_dt'] = pd.to_datetime(end_dt)
df_stock_code.rename(columns = {"ListingDate":"in_dt"}, inplace = True) 
df_stock_code = df_stock_code.sort_values(by=['code', 'base_dt'], axis=0, ascending=[True, False])  
df_stock_code = df_stock_code.drop_duplicates(['code'], keep='first')
df_stock_code = df_stock_code[['code','code_nm','in_yn','market','out_dt','out_desc','base_dt']]
df_stock_code = df_stock_code[df_stock_code['code']=='005930']

df_price = pd.read_csv('주가파일.csv', encoding='utf-8')
df_price['dt'] = pd.to_datetime(df_price['dt'])
df_price['dt_base'] = pd.to_datetime(df_price['dt_base'])
df_price['code'] = df_price['code'].astype(str)
df_price['code'] = df_price['code'].str.zfill(6)

df_usdkrw = pd.read_csv('환율파일.csv', encoding='utf-8')    
df_index = pd.read_csv('주가지수.csv', encoding='utf-8')
df_index['dt'] = pd.to_datetime(df_index['dt'])
df_usdkrw['dt'] = pd.to_datetime(df_usdkrw['dt'])

## 학습 대상 종목 선정 
stock_list = pd.read_csv("./stock_list.csv")
stock_list['종목코드'] = stock_list['종목코드'].apply(lambda x : str(x).zfill(6))
stock_list.rename(columns = {"종목코드":"code"}, inplace = True) 

df_master = df_price[(df_price['open'] > 0) & (df_price['end'] > 0) & (df_price['cnt'] > 0)]
df_master['amt']  = df_master['end'] * df_master['cnt']   
df_amt = df_master.groupby(['code'], as_index=False)[['end', 'cnt', 'amt']].mean()
df_amt = df_amt[(df_amt['end'] >= 3000) & (df_amt['amt'] >= 2000000000)]     
dl_amt = df_amt['code'].values.tolist() 
dl_200 = stock_list['code'].values.tolist() 
dl_amt = set(dl_amt + dl_200)
df_master = df_master[df_master['code'].isin(dl_amt)]   

## Target 생성
df_master = df_master.sort_values(by=['code', 'dt'], axis=0, ascending=[True, False])  
df_master['num'] = df_master.groupby('code')['dt'].rank(ascending=True).astype(int)
df_master['y_01d_yn']  = np.where(df_master['code'] == df_master['code'].shift(1), 1, 0)  
df_master['y_02d_yn']  = np.where(df_master['code'] == df_master['code'].shift(2), 1, 0)  
df_master['y_03d_yn']  = np.where(df_master['code'] == df_master['code'].shift(3), 1, 0)  
df_master['y_04d_yn']  = np.where(df_master['code'] == df_master['code'].shift(4), 1, 0)  
df_master['y_05d_yn']  = np.where(df_master['code'] == df_master['code'].shift(5), 1, 0)  
df_master['y_rt_01d_eeup']  = df_master['end'].shift(1) / df_master['end']
df_master['y_rt_02d_eeup']  = df_master['end'].shift(2) / df_master['end']
df_master['y_rt_03d_eeup']  = df_master['end'].shift(3) / df_master['end']
df_master['y_rt_04d_eeup']  = df_master['end'].shift(4) / df_master['end']
df_master['y_rt_05d_eeup']  = df_master['end'].shift(5) / df_master['end']

df_master = df_master.sort_values(by=['code', 'dt'], axis=0) 
df_master = pd.merge(df_master, df_index[['dt', 'end_kpi', 'end_ksd']], how='left', on='dt')
df_master = pd.merge(df_master, df_stock_code[['code', 'market']], how='left', on='code')
df_master['x_rt_end_index']  = np.where(df_master['market'] == 'KOSPI', df_master['end'] / df_master['end_kpi'], 
                                        df_master['end'] / df_master['end_ksd'])
df_master = pd.merge(df_master, df_usdkrw[['dt', 'end_usdkrw']], how='left', on='dt')
df_master['x_rt_end_usdkrw'] = df_master['end'] / df_master['end_usdkrw']

## 시/저/고/저 파생
df_master['avg_amt_20'] = df_master['amt'].rolling(window=20).mean() 
df_master['x_rt_h_l']  = np.where(df_master['low'] == 0,  np.nan, df_master['high'] / df_master['low'])      
df_master['x_rt_e_s']  = np.where(df_master['open'] == 0,  np.nan, df_master['end'] / df_master['open'])   
df_master['x_rt_e_l']  = np.where(df_master['low'] == 0,  np.nan, df_master['end'] / df_master['low']) 
df_master['x_rt_h_s']  = np.where(df_master['open'] == 0,  np.nan, df_master['high'] / df_master['open'])   
df_master['x_rt_s_l']  = np.where(df_master['low'] == 0,  np.nan, df_master['open'] / df_master['low'])  
df_master['x_rt_bf_1']  = np.where(df_master['end'].shift(1) == 0,   np.nan, df_master['end'] / df_master['end'].shift(1))

## 과거 평균 대비 이격도 Feature 
k  = 3; k2 = 10; k3 = 40; k4 = 80

dl_num_cols = ['open', 'high', 'low', 'end', 'amt', 'end_kpi', 'end_ksd', 'x_rt_end_index',
               'x_rt_bf_1', 'x_rt_e_s', 'x_rt_e_l', 'end_usdkrw', 'x_rt_end_usdkrw' ] 

dl_cols =[('x_rt_bf_1_' + w) for w in dl_num_cols]
df_master[dl_cols]  = np.where(df_master[dl_num_cols].shift(1) == 0,  np.nan, df_master[dl_num_cols] / df_master[dl_num_cols].shift(1))

dl_cols =[('x_rt_ma_k_' + w) for w in dl_num_cols]
df_master[dl_cols]  = np.where(df_master[dl_num_cols].rolling(window=k).mean() == 0,  np.nan, 
                      (df_master[dl_num_cols]-df_master[dl_num_cols].rolling(window=k).mean()) / df_master[dl_num_cols].rolling(window=k2).mean())  
df_master[dl_cols] = np.where(df_master[dl_num_cols] >= df_master[dl_num_cols].rolling(window=k).mean(), 
                      abs(df_master[dl_cols]), -abs(df_master[dl_cols]))                                                
                        
dl_cols =[('x_rt_ma_k2_' + w) for w in dl_num_cols]
df_master[dl_cols]  = np.where(df_master[dl_num_cols].rolling(window=k2).mean() == 0,  np.nan, 
                      (df_master[dl_num_cols]-df_master[dl_num_cols].rolling(window=k2).mean()) / df_master[dl_num_cols].rolling(window=k2).mean())  
df_master[dl_cols] = np.where(df_master[dl_num_cols] >= df_master[dl_num_cols].rolling(window=k2).mean(), 
                      abs(df_master[dl_cols]), -abs(df_master[dl_cols]))
                        
dl_cols =[('x_rt_ma_k_k2_' + w) for w in dl_num_cols]
df_master[dl_cols]  = np.where(df_master[dl_num_cols].rolling(window=k2).mean() == 0,  np.nan, 
                      (df_master[dl_num_cols].rolling(window=k).mean()-df_master[dl_num_cols].rolling(window=k2).mean()) / df_master[dl_num_cols].rolling(window=k2).mean())  
df_master[dl_cols] = np.where(df_master[dl_num_cols].rolling(window=k).mean() >= df_master[dl_num_cols].rolling(window=k2).mean(), 
                      abs(df_master[dl_cols]), -abs(df_master[dl_cols]))

dl_cols =[('x_rt_ma_k3_' + w) for w in dl_num_cols]
df_master[dl_cols]  = np.where(df_master[dl_num_cols].rolling(window=k3).mean() == 0,  np.nan, 
							 (df_master[dl_num_cols]-df_master[dl_num_cols].rolling(window=k3).mean()) / df_master[dl_num_cols].rolling(window=k2).mean())  
df_master[dl_cols] = np.where(df_master[dl_num_cols] >= df_master[dl_num_cols].rolling(window=k3).mean(), 
							 abs(df_master[dl_cols]), -abs(df_master[dl_cols]))
							 
dl_cols =[('x_rt_ma_k_k3_' + w) for w in dl_num_cols]
df_master[dl_cols]  = np.where(df_master[dl_num_cols].rolling(window=k3).mean() == 0,  np.nan, 
							 (df_master[dl_num_cols].rolling(window=k).mean()-df_master[dl_num_cols].rolling(window=k3).mean()) / df_master[dl_num_cols].rolling(window=k2).mean())  
df_master[dl_cols] = np.where(df_master[dl_num_cols].rolling(window=k).mean() >= df_master[dl_num_cols].rolling(window=k3).mean(), 
							 abs(df_master[dl_cols]), -abs(df_master[dl_cols]))
							 
dl_cols =[('x_rt_ma_k2_k3_' + w) for w in dl_num_cols]
df_master[dl_cols]  = np.where(df_master[dl_num_cols].rolling(window=k3).mean() == 0,  np.nan, 
							 (df_master[dl_num_cols].rolling(window=k2).mean()-df_master[dl_num_cols].rolling(window=k3).mean()) / df_master[dl_num_cols].rolling(window=k2).mean())  
df_master[dl_cols] = np.where(df_master[dl_num_cols].rolling(window=k2).mean() >= df_master[dl_num_cols].rolling(window=k3).mean(), 
							 abs(df_master[dl_cols]), -abs(df_master[dl_cols]))							 
							 
dl_cols =[('x_rt_ma_k4_' + w) for w in dl_num_cols]
df_master[dl_cols]  = np.where(df_master[dl_num_cols].rolling(window=k4).mean() == 0,  np.nan, 
							 (df_master[dl_num_cols]-df_master[dl_num_cols].rolling(window=k4).mean()) / df_master[dl_num_cols].rolling(window=k2).mean())  
df_master[dl_cols] = np.where(df_master[dl_num_cols] >= df_master[dl_num_cols].rolling(window=k4).mean(), 
							 abs(df_master[dl_cols]), -abs(df_master[dl_cols]))
							 
dl_cols =[('x_rt_ma_k_k4_' + w) for w in dl_num_cols]
df_master[dl_cols]  = np.where(df_master[dl_num_cols].rolling(window=k4).mean() == 0,  np.nan, 
							 (df_master[dl_num_cols].rolling(window=k).mean()-df_master[dl_num_cols].rolling(window=k4).mean()) / df_master[dl_num_cols].rolling(window=k2).mean())  
df_master[dl_cols] = np.where(df_master[dl_num_cols].rolling(window=k).mean() >= df_master[dl_num_cols].rolling(window=k4).mean(), 
							 abs(df_master[dl_cols]), -abs(df_master[dl_cols]))
							 
dl_cols =[('x_rt_ma_k2_k4_' + w) for w in dl_num_cols]
df_master[dl_cols]  = np.where(df_master[dl_num_cols].rolling(window=k4).mean() == 0,  np.nan, 
							 (df_master[dl_num_cols].rolling(window=k2).mean()-df_master[dl_num_cols].rolling(window=k4).mean()) / df_master[dl_num_cols].rolling(window=k2).mean())  
df_master[dl_cols] = np.where(df_master[dl_num_cols].rolling(window=k2).mean() >= df_master[dl_num_cols].rolling(window=k4).mean(), 
							 abs(df_master[dl_cols]), -abs(df_master[dl_cols]))
							 
dl_cols =[('x_rt_ma_k3_k4_' + w) for w in dl_num_cols]
df_master[dl_cols]  = np.where(df_master[dl_num_cols].rolling(window=k4).mean() == 0,  np.nan, 
							 (df_master[dl_num_cols].rolling(window=k3).mean()-df_master[dl_num_cols].rolling(window=k4).mean()) / df_master[dl_num_cols].rolling(window=k2).mean())  
df_master[dl_cols] = np.where(df_master[dl_num_cols].rolling(window=k3).mean() >= df_master[dl_num_cols].rolling(window=k4).mean(), 
							 abs(df_master[dl_cols]), -abs(df_master[dl_cols]))

## 단순이동평균 대비 지수이동평균 대비 이격도
for col in dl_num_cols:
	str_ma05 = 'ma05_' + col
	str_ema05 = 'ema05_' + col
	str_rt_ema05_ma05 = 'x_rt_ema05_ma05_' + col
	df_master[str_ma05] = df_master[col].rolling(window=5).mean() 		
	df_master[str_ema05]  = df_master[col].shift(5) 
	tmp1 = 0.7 	
	for i in range(5):
		df_master[str_ema05]  = (df_master[col].shift(5-1-i) * tmp1) + (df_master[str_ema05] * (1-tmp1))		
	df_master[str_rt_ema05_ma05] = np.where(df_master[str_ma05] == 0,  np.nan, (df_master[str_ema05]-df_master[str_ma05]) / df_master[str_ma05])  
	df_master[str_rt_ema05_ma05] = np.where(df_master[str_ema05] >= df_master[str_ma05], abs(df_master[str_rt_ema05_ma05]), -abs(df_master[str_rt_ema05_ma05]))		   

	str_ma20 = 'ma20_' + col
	str_ema20 = 'ema20_' + col
	str_rt_ema20_ma20 = 'x_rt_ema20_ma20_' + col
	df_master[str_ma20] = df_master[col].rolling(window=20).mean() 		
	df_master[str_ema20]  = df_master[col].shift(20) 
	tmp1 = 0.7 	
	for i in range(20):
		df_master[str_ema20]  = (df_master[col].shift(20-1-i) * tmp1) + (df_master[str_ema20] * (1-tmp1))		
	df_master[str_rt_ema20_ma20] = np.where(df_master[str_ma20] == 0,  np.nan, (df_master[str_ema20]-df_master[str_ma20]) / df_master[str_ma20])  
	df_master[str_rt_ema20_ma20] = np.where(df_master[str_ema20] >= df_master[str_ma20], abs(df_master[str_rt_ema20_ma20]), -abs(df_master[str_rt_ema20_ma20]))		   

	str_ma40 = 'ma40_' + col
	str_ema40 = 'ema40_' + col
	str_rt_ema40_ma40 = 'x_rt_ema40_ma40_' + col
	df_master[str_ma40] = df_master[col].rolling(window=40).mean() 		
	df_master[str_ema40]  = df_master[col].shift(40) 
	tmp1 = 0.7 	
	for i in range(40):
		df_master[str_ema40]  = (df_master[col].shift(40-1-i) * tmp1) + (df_master[str_ema40] * (1-tmp1))		
	df_master[str_rt_ema40_ma40] = np.where(df_master[str_ma40] == 0,  np.nan, (df_master[str_ema40]-df_master[str_ma40]) / df_master[str_ma40])  
	df_master[str_rt_ema40_ma40] = np.where(df_master[str_ema40] >= df_master[str_ma40], abs(df_master[str_rt_ema40_ma40]), -abs(df_master[str_rt_ema40_ma40]))		   

	str_ma80 = 'ma80_' + col
	str_ema80 = 'ema80_' + col
	str_rt_ema80_ma80 = 'x_rt_ema80_ma80_' + col
	df_master[str_ma80] = df_master[col].rolling(window=80).mean() 		
	df_master[str_ema80]  = df_master[col].shift(80) 
	tmp1 = 0.7 	
	for i in range(80):
		df_master[str_ema80]  = (df_master[col].shift(80-1-i) * tmp1) + (df_master[str_ema80] * (1-tmp1))		
	df_master[str_rt_ema80_ma80] = np.where(df_master[str_ma80] == 0,  np.nan, (df_master[str_ema80]-df_master[str_ma80]) / df_master[str_ma80])  
	df_master[str_rt_ema80_ma80] = np.where(df_master[str_ema80] >= df_master[str_ma80], abs(df_master[str_rt_ema80_ma80]), -abs(df_master[str_rt_ema80_ma80]))		   

	
## Lag 변수		
dl_code = ['x_rt_bf_1', 'x_rt_h_l', 'x_rt_e_s', 'x_rt_e_l']
for m in range(10):   
	for code in dl_code:
		str_x = 'x_lag_' + code + str(m+1) 	
		df_master[str_x] = df_master[code].shift(m+1)
	

## NVI(Negative Volume Index)

# end + cnt	
df_master['x_ind_nvi_10'] = 100
df_master['cnt_tmp'] = df_master['cnt'].shift(10)
df_master['end_tmp'] = df_master['end'].shift(10) 
for i in range(9, -1, -1):  
	df_master['tmp1'] = np.where(df_master['end_tmp'] == 0, np.nan, 
								   ((df_master['end'].shift(i) - df_master['end_tmp']) / df_master['end_tmp']) * 100)
	df_master['x_ind_nvi_10'] = np.where(df_master['cnt'].shift(i) < df_master['cnt_tmp'], df_master['x_ind_nvi_10'] + df_master['tmp1'], 
										df_master['x_ind_nvi_10'])
	df_master['cnt_tmp'] = df_master['cnt'].shift(i)
	df_master['end_tmp'] = df_master['end'].shift(i)
	
# high + cnt	
df_master['x_ind_h_nvi_10'] = 100
df_master['cnt_tmp'] = df_master['cnt'].shift(10)
df_master['high_tmp'] = df_master['high'].shift(10) 
for i in range(9, -1, -1):  
	df_master['tmp1'] = np.where(df_master['high_tmp'] == 0, np.nan, 
								   ((df_master['high'].shift(i) - df_master['high_tmp']) / df_master['high_tmp']) * 100)
	df_master['x_ind_h_nvi_10'] = np.where(df_master['cnt'].shift(i) < df_master['cnt_tmp'], df_master['x_ind_h_nvi_10'] + df_master['tmp1'], 
										df_master['x_ind_h_nvi_10'])
	df_master['cnt_tmp'] = df_master['cnt'].shift(i)
	df_master['high_tmp'] = df_master['high'].shift(i)
	

# low + cnt	
df_master['x_ind_l_nvi_10'] = 100
df_master['cnt_tmp'] = df_master['cnt'].shift(10)
df_master['low_tmp'] = df_master['low'].shift(10) 
for i in range(9, -1, -1):  
	df_master['tmp1'] = np.where(df_master['low_tmp'] == 0, np.nan, 
								   ((df_master['low'].shift(i) - df_master['low_tmp']) / df_master['low_tmp']) * 100)
	df_master['x_ind_l_nvi_10'] = np.where(df_master['cnt'].shift(i) < df_master['cnt_tmp'], df_master['x_ind_l_nvi_10'] + df_master['tmp1'], 
										df_master['x_ind_l_nvi_10'])
	df_master['cnt_tmp'] = df_master['cnt'].shift(i)
	df_master['low_tmp'] = df_master['low'].shift(i)
		
	
## PVI(Positive Volume Index)

# end + cnt										
df_master['x_ind_pvi_10'] = 100
df_master['cnt_tmp'] = df_master['cnt'].shift(10) 
df_master['end_tmp'] = df_master['end'].shift(10) 
for i in range(9, -1, -1):  
	df_master['tmp1'] = np.where(df_master['end_tmp'] == 0, np.nan, 
								   ((df_master['end'].shift(i) - df_master['end_tmp']) / df_master['end_tmp']) * 100)
	df_master['x_ind_pvi_10'] = np.where(df_master['cnt'].shift(i) > df_master['cnt_tmp'], df_master['x_ind_pvi_10'] + df_master['tmp1'], 
										df_master['x_ind_pvi_10'])
	df_master['cnt_tmp'] = df_master['cnt'].shift(i)
	df_master['end_tmp'] = df_master['end'].shift(i)
	
# high + cnt										
df_master['x_ind_h_pvi_10'] = 100
df_master['cnt_tmp'] = df_master['cnt'].shift(10) 
df_master['high_tmp'] = df_master['high'].shift(10) 
for i in range(9, -1, -1):  
	df_master['tmp1'] = np.where(df_master['high_tmp'] == 0, np.nan, 
								   ((df_master['high'].shift(i) - df_master['high_tmp']) / df_master['high_tmp']) * 100)
	df_master['x_ind_h_pvi_10'] = np.where(df_master['cnt'].shift(i) > df_master['cnt_tmp'], df_master['x_ind_h_pvi_10'] + df_master['tmp1'], 
										df_master['x_ind_h_pvi_10'])
	df_master['cnt_tmp'] = df_master['cnt'].shift(i)
	df_master['high_tmp'] = df_master['high'].shift(i)
	
# low + cnt										
df_master['x_ind_l_pvi_10'] = 100
df_master['cnt_tmp'] = df_master['cnt'].shift(10) 
df_master['low_tmp'] = df_master['low'].shift(10) 
for i in range(9, -1, -1):  
	df_master['tmp1'] = np.where(df_master['low_tmp'] == 0, np.nan, 
								   ((df_master['low'].shift(i) - df_master['low_tmp']) / df_master['low_tmp']) * 100)
	df_master['x_ind_l_pvi_10'] = np.where(df_master['cnt'].shift(i) > df_master['cnt_tmp'], df_master['x_ind_l_pvi_10'] + df_master['tmp1'], 
										df_master['x_ind_l_pvi_10'])
	df_master['cnt_tmp'] = df_master['cnt'].shift(i)
	df_master['low_tmp'] = df_master['low'].shift(i)

## Rule Feature
df_master['x_cls_01'] = np.where((df_master['open'] < df_master['end']) & (df_master['high'] > df_master['end']) & (df_master['high']/df_master['end'].shift(1) <=1.01), 1,
					    np.where((df_master['open'] < df_master['end']) & (df_master['high'] > df_master['end']), 2, 3))
						
dl_list = df_master.columns[pd.Series(df_master.columns).str.startswith('x_cls_')]   
df_master[dl_list] = df_master[dl_list].astype(str)     

## 분석 모집단 생성
df_mst = df_master.query("num >=101 & cnt > 0 ")	
dl_col_1 = ['dt', 'code', 'amt']	
dl_col_2 = df_mst.columns[pd.Series(df_mst.columns).str.startswith('y_')].values.tolist()   
dl_x1 = df_mst.columns[pd.Series(df_mst.columns).str.startswith('x_')].values.tolist()  
dl_x2 = ['end', 'avg_amt_20', 'end_usdkrw']  
dl_x = dl_x1 + dl_x2 
df_mst = df_mst[dl_col_1 + dl_col_2 + dl_x]

## 학습 조건 설정
dl_look_after = [1, 2, 3, 4, 5]                        # 예측기간
dl_n_trin = [800, 400, 200, 100, 50]     # 학습 데이타건수
dl_target = ['y_rt_01d_eeup', 'y_rt_02d_eeup', 'y_rt_03d_eeup', 'y_rt_04d_eeup', 'y_rt_05d_eeup']   # Target Name
dl_yn     = ['y_01d_yn', 'y_02d_yn', 'y_03d_yn', 'y_04d_yn', 'y_05d_yn']                            # Target 불능 여부

df_m2_pre_c1_tot = pd.DataFrame()
df_m2_pre_c2_tot = pd.DataFrame()

## 예측기간별 모델링
for look in dl_look_after:

	print("\n ===> look_after : ",  look)
	
	df_mst_tt = df_mst.copy()

	n_trin_sample = dl_n_trin[look-1]   
	random.seed(123)
	
	str_target            = dl_target[look-1]      
	str_yn                = dl_yn[look-1]     
	look_after            = look      
	df_mst_tt.rename(columns = {str_target:'Target'}, inplace = True)    
	
	## 학습 및 예측 조건	
	df_m1_tt_c1 = df_mst_tt[(df_mst_tt['dt'] >= '20120101') & (df_mst_tt['dt'] < pub_base_dt)]   	           
	df_m1_tt_c2 = df_mst_tt[(df_mst_tt['dt'] >= '20120101') & (df_mst_tt['dt'] < pri_base_dt)]   
	df_m2_tt_c1 = df_mst_tt[(df_mst_tt['dt'] == pub_base_dt)]  
	df_m2_tt_c2 = df_mst_tt[(df_mst_tt['dt'] == pri_base_dt)]  
	df_m1_tt_c1 = df_m1_tt_c1[df_m1_tt_c1[str_yn]==1]   
	df_m1_tt_c2 = df_m1_tt_c2[df_m1_tt_c2[str_yn]==1]   	
	df_m2_tt_c1 = df_m2_tt_c1[df_m2_tt_c1['code'].isin(stock_list['code'])] 
	df_m2_tt_c2 = df_m2_tt_c2[df_m2_tt_c2['code'].isin(stock_list['code'])] 
	
	## Target Scaling	
	target_out1 = df_m1_tt_c1['Target'].describe(percentiles = [0.2, 0.8])
	target_out2 = df_m1_tt_c2['Target'].describe(percentiles = [0.2, 0.8])
	df_m1_tt_c1 = df_m1_tt_c1[(df_m1_tt_c1['Target'] >= target_out1['20%']) & (df_m1_tt_c1['Target'] < target_out1['80%'])]
	df_m1_tt_c2 = df_m1_tt_c2[(df_m1_tt_c2['Target'] >= target_out2['20%']) & (df_m1_tt_c2['Target'] < target_out2['80%'])]
	

	def f_featuring(df_m1_tt, df_m2_tt):
		
		random.seed(123)   
		
		## Sampling		
		rows = random.sample(df_m1_tt.index.tolist(), n_trin_sample)     
		df_m1_sam = df_m1_tt.loc[rows]			
		df_m1_sam = df_m1_sam.sort_values(by=['dt'], axis=0)    		
		df_m2_sam = df_m2_tt.sort_values(by=['dt'], axis=0)   
		
		## X, Y 분리			
		dl_y = df_m1_sam.columns[pd.Series(df_m1_sam.columns).str.startswith('Target')].values.tolist()  			
		df_m1_x = df_m1_sam[dl_x]	
		df_m2_x = df_m2_sam[dl_x]	
		df_m1_y = df_m1_sam[dl_y]	
		df_m2_y = df_m2_sam[dl_y]
			
		## Missing 처리
		df_m1_x = df_m1_x.replace([np.inf, -np.inf], np.nan)	
		df_m2_x = df_m2_x.replace([np.inf, -np.inf], np.nan)
		df_m1_x.dropna(how='any', axis=1, inplace=True)  
		dl_tot_cols = df_m1_x.columns                    
		df_m2_x = df_m2_x[dl_tot_cols]                   
			
		dl_num_cols = df_m1_x._get_numeric_data().columns                    
		if len(dl_num_cols) >= 1:
			imp = SimpleImputer(missing_values=np.nan, strategy='median')   
			imp = imp.fit(df_m1_x[dl_num_cols])
			df_m1_x[dl_num_cols] = imp.transform(df_m1_x[dl_num_cols])
			df_m2_x[dl_num_cols] = imp.transform(df_m2_x[dl_num_cols])
			
		dl_cls_cols = df_m1_x.select_dtypes(include=['object']).columns.tolist()		
		if len(dl_cls_cols) >= 1:		
			df_m1_x[dl_cls_cols] = df_m1_x[dl_cls_cols].fillna('missing')
			df_m2_x[dl_cls_cols] = df_m2_x[dl_cls_cols].fillna('missing')

		## Unique 변수 제외
		df_col_list = df_m1_x.apply(pd.Series.nunique) != 1    
		df_m1_x = df_m1_x.loc[:, df_col_list]
		df_m2_x = df_m2_x.loc[:, df_col_list]				

		## 변수 Log 변환 		
		dl_num_cols = df_m1_x._get_numeric_data().columns   
		for col in dl_num_cols:		  
			df_m1_x[col] = np.where(df_m1_x[col] < 0, -np.log(1 + -df_m1_x[col]), np.log(1 + df_m1_x[col]))   
			df_m2_x[col] = np.where(df_m2_x[col] < 0, -np.log(1 + -df_m2_x[col]), np.log(1 + df_m2_x[col]))   
					
		## 변수 표준화 
		dl_num_cols = df_m1_x._get_numeric_data().columns   
		scaler = StandardScaler().fit(df_m1_x[dl_num_cols])           
		df_m1_x[dl_num_cols] = scaler.transform(df_m1_x[dl_num_cols])  
		df_m2_x[dl_num_cols] = scaler.transform(df_m2_x[dl_num_cols])  
		
		## One Hot Encoding 		
		dl_cls_cols = df_m1_x.select_dtypes(include=['object']).columns.tolist()		
		df_m1_x_dumy = pd.get_dummies(df_m1_x[dl_cls_cols], prefix=dl_cls_cols)   
		df_m2_x_dumy = pd.get_dummies(df_m2_x[dl_cls_cols], prefix=dl_cls_cols)
		dl_col_m1 = df_m1_x_dumy.columns	
		dl_col_m2 = df_m2_x_dumy.columns

		dl_col_m1_m2 = list(set(dl_col_m1) - set(dl_col_m2))  
		if len(dl_col_m1_m2) >= 1:
			for col in dl_col_m1_m2:
				df_m2_x_dumy[col] = 0					

		df_m1_x = pd.concat([df_m1_x[dl_num_cols], df_m1_x_dumy], axis=1)
		df_m2_x = pd.concat([df_m2_x[dl_num_cols], df_m2_x_dumy], axis=1)	
		df_m1_x, df_m2_x = df_m1_x.align(df_m2_x, join='left', axis=1, fill_value=0)

		## 상관분석
		corr_matrix = df_m1_x.corr()		
		df_corr_upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))  
		to_drop = [column for column in df_corr_upper.columns if (any(df_corr_upper[column] >= 0.95) | any(df_corr_upper[column] < -0.95))]   
		df_m1_x.drop(to_drop, axis=1, inplace=True)
		df_m2_x.drop(to_drop, axis=1, inplace=True)
		
		## Feature 안정성 분석		
		if len(df_m1_x.columns) > 120:   
			num = 120                    
		else:
			num = len(df_m1_x.columns)   
		model = SelectKBest(f_regression, k= num)                
		col_list = pd.DataFrame(df_m1_x.columns, columns=['col'])
		int_num = int(len(df_m1_x)/3)
		fit = model.fit(df_m1_x[:int_num], df_m1_y[:int_num])
		score_list_1 = pd.DataFrame(model.scores_, columns=['score'])
		fit = model.fit(df_m1_x[int_num:int_num*2], df_m1_y  [int_num:int_num*2])
		score_list_2 = pd.DataFrame(model.scores_, columns=['score'])
		fit = model.fit(df_m1_x[int_num*2:], df_m1_y[int_num*2:])
		score_list_3 = pd.DataFrame(model.scores_, columns=['score'])	
		df_importance = pd.concat([col_list, score_list_1, score_list_2, score_list_3], axis=1)     
		df_importance['mean'] = df_importance.iloc[:, [1, 2, 3]].min(1)
		df_importance['rank'] = df_importance['mean'].rank(ascending=0)  
		df_m1_x = df_m1_x[df_importance.loc[df_importance['rank'] <= num, 'col']]
		df_m2_x = df_m2_x[df_importance.loc[df_importance['rank'] <= num, 'col']]		
						
		return df_m1_x, df_m2_x, df_m1_y, df_m2_y, df_m2_sam
		
	df_m1_x_c1, df_m2_x_c1, df_m1_y_c1, df_m2_y_c1, df_m2_sam_c1 = f_featuring(df_m1_tt_c1, df_m2_tt_c1)
	df_m1_x_c2, df_m2_x_c2, df_m1_y_c2, df_m2_y_c2, df_m2_sam_c2 = f_featuring(df_m1_tt_c2, df_m2_tt_c2)

	## Model Fitting		
	gbl = globals()
	def f_modeling_1(case_flg):

		df_m1_x = gbl['df_m1_x_' + case_flg]
		df_m2_x = gbl['df_m2_x_' + case_flg]
		df_m1_y = gbl['df_m1_y_' + case_flg]
		df_m2_y = gbl['df_m2_y_' + case_flg]	
		df_m2_sam = gbl['df_m2_sam_' + case_flg]
		train_data = lgb.Dataset(df_m1_x, label = df_m1_y)	
		parameters = {
			'n_leaves':10,
			'n_estimators':80, 
			'max_depth':-1,  
			'learning_rate':0.01, 
			'subsample':1,
			'colsample_bytree':0.8,
			'reg_alpha':0.1,
			'reg_lambda':1,
			'objective': 'regression',
			'min_data':1,
			'min_data_in_bin':1,	
			'random_state' : 123} 			
		fit = lgb.train(parameters, train_data, num_boost_round=5000)	
		df_m1_pre = pd.DataFrame(fit.predict(df_m1_x))
		df_m2_pre = pd.DataFrame(fit.predict(df_m2_x))
							
		## Predict
		df_m2_pre.rename(columns = {0:"score"}, inplace = True)
		df_m2_pre = pd.concat([df_m2_pre.reset_index(drop=True), 
							   df_m2_sam[['Target', 'dt', 'code', 'amt', 'end']].reset_index(drop=True)], axis=1)		
		df_m2_pre['end_a1'] = df_m2_pre['end'] * df_m2_pre['Target']			
		df_m2_pre['end_a1_pred'] = df_m2_pre['end'] * df_m2_pre['score'] 
		df_m2_pre['model'] = 'M1' 
		df_m2_pre['case'] = case_flg
		df_m2_pre['look_after'] = look_after

		return df_m2_pre

	df_m2_pre_c1 = f_modeling_1('c1')       
	df_m2_pre_c2 = f_modeling_1('c2')	
	df_m2_pre_c1_tot = df_m2_pre_c1_tot.append(df_m2_pre_c1)
	df_m2_pre_c2_tot = df_m2_pre_c2_tot.append(df_m2_pre_c2)


 ===> look_after :  1
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002000 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 29841
[LightGBM] [Info] Number of data points in the train set: 800, number of used features: 120
[LightGBM] [Info] Start training from score 1.000430
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002000 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 29841
[LightGBM] [Info] Number of data points in the train set: 800, number of used features: 120
[LightGBM] [Info] Start training from score 1.000730

 ===> look_after :  2
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning

# 2. 모델 후처리 및 제출

Stage 5에서 LGBM 모델에 적절한 데이터를 넣어 학습시켰고, 우리가 필요로 하는 Public 날짜와 Private 날짜의 주가를 예측했습니다.

메인이 되는 모델링은 끝났고, 최종적으로 파일을 제출하기 위해 후처리 작업을 통해 데이터를 다듬도록 하겠습니다.



In [4]:
# 기준일 종가 추출

df_end_pub = df_price[(df_price['dt'] == pub_base_dt)]
df_end_pub = df_end_pub[df_end_pub['code'].isin(stock_list['code'])] 
df_end_pri = df_price[(df_price['dt'] == pri_base_dt)]
df_end_pri = df_end_pri[df_end_pri['code'].isin(stock_list['code'])] 


Public 기준일, Private 기준일의 삼성전자 종가를 따로 빼어내, df_end_pub과 df_end_pri에 저장하였습니다.

In [5]:
# Public 후처리
df_submit_pub = df_m2_pre_c1_tot[['code', 'dt', 'end_a1_pred', 'look_after']]
df_submit_pub = pd.merge(stock_list[['code']], df_submit_pub, how='left', on=['code'])
df_submit_pub = pd.merge(df_submit_pub, df_end_pub[['code', 'dt', 'end']], how='left', on=['code'])
df_submit_pub['end_a1_pred'] = np.where(df_submit_pub['end_a1_pred'].isnull(), df_submit_pub['end'], df_submit_pub['end_a1_pred'])  # 거래정지
df_submit_pub['look_after']  = np.where(df_submit_pub['look_after'].isnull(), 1, df_submit_pub['look_after'])  
df_submit_pub = pd.pivot_table(df_submit_pub, index='look_after', columns='code', values='end_a1_pred')
df_submit_pub['Day'] = dl_predict_pub_day
cols = df_submit_pub.columns.tolist()                  
cols = cols[-1:] + cols[:-1]
df_submit_pub = df_submit_pub[cols] 	                
df_submit_pub = df_submit_pub.fillna(method='ffill')  	

**pd.pivot_table()** 함수가 조금 생소하실 수도 있을 것 같습니다.</br>
**pivot_table()** 메소드는 피벗테이블을 생성하는 메소드로, 엑셀을 사용하신다면 한 번쯤 사용하셨을 수도 있을텐데요.


**피벗(pivot)** 의 사전적 의미에 대해서 알아보면 피벗은 ‘회전 축’이라는 뜻을 가지고 있는데, 피벗 테이블도 축과 연관지어서 생각해볼 수 있습니다. 

우리가 표로 정리해서 보고자 하는 관점의 축을 잡고, 표로 만들어 주는 기능이 바로 **피벗 테이블**이라고 볼 수 있습니다.

예를 들어, 우리가 특정 기간에 판매한 내역이 있다면 구매가 발생되는 요인을 파악하기 위해서 어떤 고객이 가장 많이 우리 매장에 방문했는지, 어떤 품목이 제일 잘 나갔는지 등등 다양한 관점에서 분석을 해보고 싶을 때가 있습니다. 

그렇게 다양한 관점에서 데이터를 확인해볼 수 있는 게 바로 피벗테이블입니다.

피벗테이블은 보고 싶은 기준을 바로바로 변경해서 볼 수 있게되어 복잡하게 수식을 쓰거나 기준이 달라질 때마다 새로 작성을 할 필요가 없습니다.


In [6]:
## Private 후처리
df_submit_pri = df_m2_pre_c2_tot[['code', 'dt', 'end_a1_pred', 'look_after']]
df_submit_pri = pd.merge(stock_list[['code']], df_submit_pri, how='left', on=['code'])
df_submit_pri = pd.merge(df_submit_pri, df_end_pri[['code', 'dt', 'end']], how='left', on=['code'])
df_submit_pri['end_a1_pred'] = np.where(df_submit_pri['end_a1_pred'].isnull() & (df_submit_pri['code'] == '017670'), df_submit_pri['end']/5, # 액면분할 : sk텔레콤 5대 1
                               np.where(df_submit_pri['end_a1_pred'].isnull() & df_submit_pri['end'].notnull(), df_submit_pri['end'],        # 거래정지
                               np.where(df_submit_pri['end'].isnull(), 0, df_submit_pri['end_a1_pred'])))                                    # 상장폐지
df_submit_pri['look_after']  = np.where(df_submit_pri['look_after'].isnull(), 1, df_submit_pri['look_after'])  
df_submit_pri = pd.pivot_table(df_submit_pri, index='look_after', columns='code', values='end_a1_pred')
df_submit_pri['Day'] = dl_predict_pri_day
cols = df_submit_pri.columns.tolist()                  
cols = cols[-1:] + cols[:-1]
df_submit_pri = df_submit_pri[cols] 	                
df_submit_pri = df_submit_pri.fillna(method='ffill') 

## Public, Private 결합
df_submit = df_submit_pub.append(df_submit_pri)
df_submit = df_submit.reset_index(drop=True) 

## Submission
today = datetime.datetime.today().strftime('%Y%m%d')
str_file = './submission_' + str(today) + '.csv'
df_submit.to_csv(str_file, index=False)  


submission.csv 파일은 다음과 같은 형식으로 구성되어 있습니다.

<img src = "https://cherry-orbit-69a.notion.site/image/https%3A%2F%2Fs3-us-west-2.amazonaws.com%2Fsecure.notion-static.com%2Fb1f8ee9a-1cfc-4df2-90c1-3ed651a3ad97%2FUntitled.png?id=8302014c-94f0-48d6-91f8-4f7b4badf090&table=block&spaceId=9b3f14ce-6c75-4c95-8989-493ed6e93a8a&width=700&userId=&cache=v2">


현재와 달라진 주식 종목 상황을 맞춰주고, 위의 형식을 맞춰주기 위해 코드를 적용했습니다.

우리는 삼성전자(005930) 종목만 모델링을 진행했기 때문에, 저장한 파일에는 다음과 같은 형식으로 저장되어 있을 것입니다.

<img src = "https://cherry-orbit-69a.notion.site/image/https%3A%2F%2Fs3-us-west-2.amazonaws.com%2Fsecure.notion-static.com%2F7c7f8a02-02f5-4f12-a430-c47bf427f665%2FUntitled.png?id=12b6d9a2-9969-40d2-bb3b-db6f4f91d17f&table=block&spaceId=9b3f14ce-6c75-4c95-8989-493ed6e93a8a&width=420&userId=&cache=v2">




데이커 여러분이 대회를 참여하게 된다면, 이제 이 파일을 대회의 **“제출”** 탭에 업로드하면 제출까지의 모든 과정이 마무리되게 됩니다.

제출 파일을 업로드하면, 파일 검수 및 스코어 계산까지 약간의 시간이 소요되고 리더보드에서 여러분의 점수를 확인할 수 있을거에요!


# 3. 마무리


축하드립니다. 🎉</br>
여러분은 이제 삼성전자의 주식을 예측할 수 있는 사람이 되었습니다. 

모든 스테이지를 저희 팀과 함께해주셔서 감사합니다! 


이 경험을 기반으로 다양한 주식 데이터를 활용해 주가의 향방을 예측할 수 있는 데이커가 되셨으면 좋겠습니다!

이상으로 6단계에 걸친 **'주식 종가 예측 경진대회'** 의 PBL 학습을 완료하겠습니다!


🐨 코알라팀 🐨

